# Importing Libraries
Import libraries and create the dataframe using pandas.

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline, make_pipeline

from google.colab import drive
drive.mount('/content/drive')


# Insert the directory and import the myanfis module
import sys
sys.path.insert(0,'/content/drive/My Drive/Biogas RMS project/Regression models/ANFIS/AnfisTensorflow2.0/Models')
import myanfis

from sklearn.metrics import r2_score

df = pd.read_csv('/content/drive/MyDrive/Biogas RMS project/Datasets/Paper 13/paper13.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Biomass type,Reactor/feeding,VS (%),pH,OLR (g VS/l.d),HRT (d),T (°C),Reactor Volume (m³),Cumulated biogas volume (L/(g VS))
0,0,0,10.00,7.62,0.6270,19.2,55,0.0500,0.0668
1,0,2,15.30,8.00,3.1702,47.0,37,0.0473,0.6765
2,0,0,4.78,7.25,1.2400,15.0,37,0.0450,0.8227
3,0,0,4.78,7.25,1.7600,15.0,37,0.0450,0.6219
4,0,2,6.36,7.30,3.2000,25.0,35,0.0400,0.5755


# Dropping columns
As the model can work only with upto 6 input columns

In [32]:
df.drop(columns=['Biomass type', 'Reactor Volume (m³)', 'T (°C)', 'pH', 'OLR (g VS/l.d)', 'VS (%)'], inplace=True)
df.head()

,Reactor/feeding,HRT (d),Cumulated biogas volume (L/(g VS))
0,0,19.2,0.0668
1,2,47.0,0.6765
2,0,15.0,0.8227
3,0,15.0,0.6219
4,2,25.0,0.5755


## Splitting the data into Test and training data

In [33]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['Cumulated biogas volume (L/(g VS))'])
y = df['Cumulated biogas volume (L/(g VS))']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)

# Pipeline information

(Since this data does not have missing values, not imputation is required.)

The following transformers will be applied in the pipeline.

Step 1: OneHotEncoding the categorical columns. 

Step 2: Scaling using StandardScaler





In [ ]:
# 1. One Hot Encoding
# trf0 = ColumnTransformer(transformers = [
#     ('ohe0', OneHotEncoder(drop='first'), ['Biomass type','Reactor/feeding'])
# ], remainder='passthrough')

In [37]:
# 2. Scaling
#  Scaling only the temperature column
trf2 = ColumnTransformer([
    ('scale', StandardScaler(), slice(0,10))
], remainder = 'passthrough')

## Creating the pipeline

In [38]:
pipe = Pipeline([
    # ('ohe0', trf0),
    ('scaling', trf2)
])

# Display Pipeline

from sklearn import set_config
set_config(display='diagram')
# Show the steps involved in the pipeline
pipe.named_steps


{'scaling': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', StandardScaler(),
                                  slice(0, 10, None))])}

## Transforming data using pipeline

In [39]:
# Visualize 
X_trans = pipe.fit(X)
X_train_trans = pipe.transform(X_train)
X_test_trans = pipe.transform(X_test)
X_pd = pd.DataFrame(X_test_trans)
X_pd.head()

,0,1
0,0.915494,0.114379
1,-0.153407,-0.529037
2,-0.153407,-0.529037
3,1.449944,-0.207329
4,-1.222308,1.401212


# Download the train test split

In [ ]:
# # Training data
# y_train_arr = np.transpose(y_train.to_numpy())
# y_train_arr  = y_train_arr.reshape(y_train.shape[0],1)
# train_data = np.append(X_train_trans,y_train_arr, axis=1)
# pd.DataFrame(train_data).to_csv('train_data.txt', sep='\t')

# # Test data
# y_test_arr = np.transpose(y_test.to_numpy())
# y_test_arr  = y_test_arr.reshape(y_test.shape[0],1)
# test_data = np.append(X_test_trans,y_test_arr, axis=1)
# pd.DataFrame(test_data).to_csv('test_data.txt', sep='\t')

# Setting the ANFIS parameters

In [59]:
param = myanfis.fis_parameters(
        n_input = 2,                # no. of Regressors
        n_memb = 2,                 # no. of fuzzy memberships
        batch_size = 1,            # 16 / 32 / 64 / ...
        memb_func = 'gaussian',      # 'gaussian' / 'gbellmf' / 'sigmoid'
        optimizer = 'sgd',          # sgd / adam / ...
        loss = tf.keras.losses.MeanAbsoluteError(),               # mse / mae / huber_loss / mean_absolute_percentage_error / ...
        n_epochs = 15               # 10 / 25 / 50 / 100 / ...
        )

# Compiling the model

In [60]:
fis = myanfis.ANFIS(n_input = param.n_input,
                    n_memb = param.n_memb,
                    batch_size = param.batch_size,
                    memb_func = param.memb_func,
                    name = 'firstAnfis' # buradaki ismi değiştirin
                    )

fis.model.compile(optimizer=param.optimizer,
                    loss=param.loss,
                    metrics=['mae']  # ['mae', 'mse']
                    )


# Fitting the model

In [61]:
 history = fis.fit(X_train_trans, y_train,
                  epochs=param.n_epochs,
                  batch_size=param.batch_size,
                  )
######## Predicting Results #########
y_pred = fis.model.predict(X_test_trans)
r2_score(y_test, y_pred)

Epoch 1/15
86/86 [==============================] - 1s 2ms/step - loss: 1.5580 - mae: 1.5580
Epoch 2/15
86/86 [==============================] - 0s 1ms/step - loss: 1.1594 - mae: 1.1594
Epoch 3/15
86/86 [==============================] - 0s 2ms/step - loss: 0.9806 - mae: 0.9806
Epoch 4/15
86/86 [==============================] - 0s 2ms/step - loss: 0.8525 - mae: 0.8525
Epoch 5/15
86/86 [==============================] - 0s 2ms/step - loss: 0.7681 - mae: 0.7681
Epoch 6/15
86/86 [==============================] - 0s 2ms/step - loss: 0.6895 - mae: 0.6895
Epoch 7/15
86/86 [==============================] - 0s 2ms/step - loss: 0.6382 - mae: 0.6382
Epoch 8/15
86/86 [==============================] - 0s 2ms/step - loss: 0.6101 - mae: 0.6101
Epoch 9/15
86/86 [==============================] - 0s 2ms/step - loss: 0.5726 - mae: 0.5726
Epoch 10/15
86/86 [==============================] - 0s 2ms/step - loss: 0.5277 - mae: 0.5277
Epoch 11/15
86/86 [==============================] - 0s 2ms/step - lo

InvalidArgumentError: ignored